<a href="https://colab.research.google.com/github/bighit2020/data-science-handbook/blob/master/rl_practice_1_1_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# [실습 목차]

|과정 1. 강화학습 기초 - 기본 원리 및 이론   |과정 2 . 강화학습 응용 - 학습 유형별 알고리즘 및 응용  |
|---|---|
|[실습 1] [강화 학습 실습 환경 소개](https://colab.research.google.com/drive/1fbGjNPO4GDIzUOWfMZelnDOzrW0mHl-U?usp=sharing)                 |[실습 1] [몬테카를로 학습 에이전트 개발](https://colab.research.google.com/drive/1LkRZfCDSo-1FaTLMJWBCLJPvkBbT7J99?usp=sharing)   |
|[실습 2] [최적 가치 및 최적 정책 산출 실험](https://colab.research.google.com/drive/1wzXuXqKM_4QLR-khg_6Oqs5t1mjXJc6T?usp=sharing)           |[실습 2] [시간차 예측을 통한 가치 추정](https://colab.research.google.com/drive/1jpfxv60AEDLEvi6XqYg7bErBmOqDx67B?usp=sharing)   |
|[실습 3] [정책 반복과 가치 반복 알고리즘 구현 및 실험](https://colab.research.google.com/drive/1jT9Zomj7eT4KcgteHSuI40QmHvJR4Fur?usp=sharing)   |[실습 3] [시간차 학습 에이전트 개발](https://colab.research.google.com/drive/1N8hnnHp4dY0sVFGz77RRGAHg9N24kvNp?usp=sharing)   |
|                                            |[실습 4] [모델 기반 학습 에이전트 개발](https://colab.research.google.com/drive/1rMKwUuJRFEzYie6K_lLu9bhAley7OE5V?usp=sharing)   |
|   |[실습 5] [Tic-Tac-Toe 강화학습 환경 구축](https://colab.research.google.com/drive/1pMYjlOL2JUjlOovl6mlEkipf4SQOjkcO?usp=sharing)   |
|   |[실습 6] [Tic-Tac-Toe 강화학습 게임 에이전트 개발 및 훈련](https://colab.research.google.com/drive/1izKrSIwOO6EINvO6QVXU1HOD_uRKxfJw?usp=sharing) |


---



# [과정 1 - 실습 1] 강화 학습 실습 환경 소개

## Lesson 1. Google Colaboratory (Colab)
- 구글에서 제공하는 브라우저 기반 Python 코딩 환경
  - 프로그램 환경 구성이 간단함
  - 간편한 공유
  - 텍스트로 구성된 노트 작성 편리
  - GPU 무료 액세스

- Colab 사용 설명
  - https://theorydb.github.io/dev/2019/08/23/dev-ml-colab/

- https://colab.research.google.com/ 접속하여 새로운 Colab 노트 생성 및 이용
  - 제공되는 컴퓨터 사양(19년 12월 기준)
    - Ubuntu 17.10
    - CPU 제논 2.3GHz
    - 메모리 13G
    - GPU: K80 또는 T4 :
    - TPU도 사용 가능

## Lesson 2. 그리드 월드 환경

In [ ]:
import random
import time
 
# -------------------------------
# |(0,0)|(0,1)|(0,2)|(0,3)|(0,4)|
# |(1,0)|(1,1)|(1,2)|(1,3)|(1,4)|
# |(2,0)|(2,1)|(2,2)|(2,3)|(2,4)|
# |(3,0)|(3,1)|(3,2)|(3,3)|(3,4)|
# |(4,0)|(4,1)|(4,2)|(4,3)|(4,4)|
# -------------------------------
 
 
class GridWorld():
    def __init__(
            self,
            height=5, width=5,        # 격자판의 크기
            start_state=(0, 0),       # 시작 상태
            terminal_states=[(4, 4)], # 종료 상태
            transition_reward=0.0,    # 일반적인 상태 전이 보상
            terminal_reward=1.0,      # 종료 상태로 이동하는 행동 수행 
                                      # 때 받는 보상
            outward_reward=0.0,       # 미로 바깥으로 이동하는 행동 수행
                                      # 때 받는 보상 
            warm_hole_states=None     # 윔홀 정의
    ):
        self.__version__ = "0.0.1"
 
        # 그리드월드의 세로 길이
        self.HEIGHT = height
 
        # 그리드월드의 가로 길이
        self.WIDTH = width
 
        self.STATES = []
        self.num_states = self.WIDTH * self.HEIGHT
 
        for i in range(self.HEIGHT):
            for j in range(self.WIDTH):
                self.STATES.append((i, j))
 
        # 터미널 상태를 상태 집합에서 제거
        for state in terminal_states:     
            self.STATES.remove(state)
 
        self.current_state = None
 
        # 모든 가능한 행동
        self.ACTION_UP = 0
        self.ACTION_DOWN = 1
        self.ACTION_LEFT = 2
        self.ACTION_RIGHT = 3
 
        self.ACTIONS = [
            self.ACTION_UP,
            self.ACTION_DOWN,
            self.ACTION_LEFT,
            self.ACTION_RIGHT
        ]
 
        # UP, DOWN, LEFT, RIGHT
        self.ACTION_SYMBOLS = ["↑", "↓", "←", "→"] 
 
        self.NUM_ACTIONS = len(self.ACTIONS)
        
        # 시작 상태 위치
        self.START_STATE = start_state
 
        # 종료 상태 위치
        self.TERMINAL_STATES = terminal_states
 
        # 웜홀 상태 위치
        self.WARM_HOLE_STATES = warm_hole_states
 
        # 일반 상태 전이 수행에 대한 보상 값
        self.transition_reward = transition_reward
 
        # 종료 상태 전이 수행에 대한 보상 값
        self.terminal_reward = terminal_reward
 
        # 그리드월드 바깥으로 상태 전이 수행해 대한 보상 값
        self.outward_reward = outward_reward
 
 
    # 환경 초기화 작업: 에이전트의 현재 위치를 START_STATE로 설정
    def reset(self):
        self.current_state = self.START_STATE
        return self.current_state
 
    # 에이전트의 현재 위치를 state로 설정
    def moveto(self, state):
        self.current_state = state
 
    # 주어진 state 상태가 웜홀 상태이면 True 반환하고, 아니면 False 반환
    def is_warm_hole_state(self, state):
        i, j = state
 
        if self.WARM_HOLE_STATES is not None \
            and len(self.WARM_HOLE_STATES) > 0:
            for warm_hole_info in self.WARM_HOLE_STATES:
                warm_hole_state = warm_hole_info[0]
                if i == warm_hole_state[0] and j == warm_hole_state[1]:
                    return True
        return False
 
    # 주어진 웜홀 상태 state에 대하여 미리 정해져 있는 다음 상태 반환
    def get_next_state_warm_hole(self, state):
        i, j = state
        next_state = None
 
        for warm_hole_info in self.WARM_HOLE_STATES:
            warm_hole_state = warm_hole_info[0]
            warm_hole_prime_state = warm_hole_info[1]
 
            if i == warm_hole_state[0] and j == warm_hole_state[1]:
                next_state = warm_hole_prime_state
                break
        return next_state
 
    # 주어진 웜홀 상태 state에 대하여 미리 정해져 있는 보상 반환
    def get_reward_warm_hole(self, state):
        i, j = state
        reward = None
 
        for warm_hole_info in self.WARM_HOLE_STATES:
            warm_hole_state = warm_hole_info[0]
            warm_hole_reward = warm_hole_info[2]
 
            if i == warm_hole_state[0] and j == warm_hole_state[1]:
                reward = warm_hole_reward
                break
 
        return reward
 
    # 주어진 상태 state와 행동 action에 대한 다음 상태 반환
    def get_next_state(self, state, action):
        i, j = state
 
        # 주어진 상태가 웜홀 상태이면 미리 정해진 다음 상태를 반환
        if self.is_warm_hole_state(state):
            next_state = self.get_next_state_warm_hole(state)
            next_i = next_state[0]
            next_j = next_state[1]
        # 주어진 상태가 종료 상태이면 현재 상태를 
        # 다음 상태로 설정하여 반환
        elif (i, j) in self.TERMINAL_STATES:
            next_i = i
            next_j = j
        # 주어진 행동 수행에 따른 다음 상태 반환
        # 주어진 행동에 의하여 그리드월드 밖으로 이동하는 경우 
        # 제자리에 멈춤
        else:
            if action == self.ACTION_UP:
                next_i = max(i - 1, 0)
                next_j = j
            elif action == self.ACTION_DOWN:
                next_i = min(i + 1, self.HEIGHT - 1)
                next_j = j
            elif action == self.ACTION_LEFT:
                next_i = i
                next_j = max(j - 1, 0)
            elif action == self.ACTION_RIGHT:
                next_i = i
                next_j = min(j + 1, self.WIDTH - 1)
            else:
                raise ValueError()
 
        return next_i, next_j
 
    # 주어진 상태 state에서 다음 상태 next_state로 이동할 경우
    # 얻는 보상 반환
    def get_reward(self, state, next_state):
        i, j = state
        next_i, next_j = next_state
 
        # 주어진 상태가 웜홀 상태이면 미리 정해진 보상 반환
        if self.is_warm_hole_state(state):
            reward = self.get_reward_warm_hole(state)
        else:
            # 다음 상태가 종료 상태이면 미리 정해진 보상 반환
            if (next_i, next_j) in self.TERMINAL_STATES:
                reward = self.terminal_reward
            else:
                # 주어진 행동에 의하여 그리드월드 밖으로 이동하는 경우 
                # 제자리에 멈추면서 미리 정해진 보상 반환
                if i == next_i and j == next_j:
                    reward = self.outward_reward
                # 일반적인 상태 전이인 경우 미리 정해진 
                # 일반 전이 보상 반환
                else:
                    reward = self.transition_reward
 
        return reward
 
    # 주어진 상태 state에서 행동 action을 수행할 때 전이되는  
    # 다음 상태 및 보상과 이에 대한 전이 확률을 반환
    def get_state_action_probability(self, state, action):
        next_i, next_j = self.get_next_state(state, action)
 
        reward = self.get_reward(state, (next_i, next_j))
        transition_prob = 1.0
 
        return (next_i, next_j), reward, transition_prob
 
    # 에이전트가 행동 action을 선택하여 환경에 적용할 때 호출하는 함수
    # 행동 action이 수행된 이후 전이된 다음 상태, 보상, 종료 유무 반환
    def step(self, action):
        next_i, next_j = self.get_next_state(
            state=self.current_state, action=action
        )
 
        reward = self.get_reward(self.current_state, (next_i, next_j))
 
        self.current_state = (next_i, next_j)
 
        if self.current_state in self.TERMINAL_STATES:
            done = True
        else:
            done = False
 
        return (next_i, next_j), reward, done, None
 
    # 그리드월드 환경을 화면에 출력
    def render(self, mode='human'):
        print(self.__str__())
 
    # 임의의 행동을 선택하여 반환
    def get_random_action(self):
        return random.choice(self.ACTIONS)
 
    # 그리드월드 환경을 문자열로 변환
    def __str__(self):
        gridworld_str = ""
        for i in range(self.HEIGHT):
            gridworld_str += "-------------------------------\n"
 
            for j in range(self.WIDTH):
                if self.current_state[0] == i and self.current_state[1] == j:
                    gridworld_str += "|  {0}  ".format("*")
                elif (i, j) == self.START_STATE:
                    gridworld_str += "|  {0}  ".format("S")
                elif (i, j) in self.TERMINAL_STATES:
                    gridworld_str += "|  {0}  ".format("G")
                elif self.WARM_HOLE_STATES and \
                     (i, j) in [state[0] for state in self.WARM_HOLE_STATES]:
                    gridworld_str += "|  {0}  ".format("W")
                else:
                    gridworld_str += "|     "
            gridworld_str += "|\n"
 
            for j in range(self.WIDTH):
                gridworld_str += "|({0},{1})".format(i, j)
 
            gridworld_str += "|\n"
 
        gridworld_str += "-------------------------------\n"
        return gridworld_str

In [ ]:
def main():
    env = GridWorld()
    env.reset()
    print("reset")
    env.render()

    done = False
    total_steps = 0
    while not done:
        total_steps += 1
        action = env.get_random_action()
        next_state, reward, done, _ = env.step(action)
        print("action: {0}, reward: {1}, done: {2}, total_steps: {3}".format(
            env.ACTION_SYMBOLS[action],
            reward, done, total_steps
        ))
        env.render()
        time.sleep(1)

In [ ]:
if __name__ == "__main__":
    main()

reset
-------------------------------
|  *  |     |     |     |     |
|(0,0)|(0,1)|(0,2)|(0,3)|(0,4)|
-------------------------------
|     |     |     |     |     |
|(1,0)|(1,1)|(1,2)|(1,3)|(1,4)|
-------------------------------
|     |     |     |     |     |
|(2,0)|(2,1)|(2,2)|(2,3)|(2,4)|
-------------------------------
|     |     |     |     |     |
|(3,0)|(3,1)|(3,2)|(3,3)|(3,4)|
-------------------------------
|     |     |     |     |  G  |
|(4,0)|(4,1)|(4,2)|(4,3)|(4,4)|
-------------------------------

action: ↑, reward: 0.0, done: False, total_steps: 1
-------------------------------
|  *  |     |     |     |     |
|(0,0)|(0,1)|(0,2)|(0,3)|(0,4)|
-------------------------------
|     |     |     |     |     |
|(1,0)|(1,1)|(1,2)|(1,3)|(1,4)|
-------------------------------
|     |     |     |     |     |
|(2,0)|(2,1)|(2,2)|(2,3)|(2,4)|
-------------------------------
|     |     |     |     |     |
|(3,0)|(3,1)|(3,2)|(3,3)|(3,4)|
-------------------------------
|     |     |

In [ ]:
# -------------------------------
# |(0,0)|(0,1)|(0,2)|(0,3)|(0,4)|
# |(1,0)|(1,1)|(1,2)|(1,3)|(1,4)|
# |(2,0)|(2,1)|(2,2)|(2,3)|(2,4)|
# |(3,0)|(3,1)|(3,2)|(3,3)|(3,4)|
# |(4,0)|(4,1)|(4,2)|(4,3)|(4,4)|
# -------------------------------

def main_warm_hole():
    A_POSITION = (0, 1)  # 임의로 지정한 특별한 상태 A 좌표
    B_POSITION = (0, 3)  # 임의로 지정한 특별한 상태 B 좌표

    A_PRIME_POSITION = (4, 1)  # 상태 A에서 임의의 행동시 도착할 위치 좌표
    B_PRIME_POSITION = (2, 3)  # 상태 B에서 임의의 행동시 도착할 위치 좌표

    env = GridWorld(
        warm_hole_states=[
            (A_POSITION, A_PRIME_POSITION, 10.0),
            (B_POSITION, B_PRIME_POSITION, 5.0)
        ]
    )

    env.reset()
    print("reset")
    env.render()

    done = False
    total_steps = 0
    while not done:
        total_steps += 1
        action = env.get_random_action()
        next_state, reward, done, _ = env.step(action)
        print("action: {0}, reward: {1}, done: {2}, total_steps: {3}".format(
            env.ACTION_SYMBOLS[action],
            reward, done, total_steps
        ))
        env.render()
        time.sleep(1)

In [ ]:
if __name__ == "__main__":
    main_warm_hole()

reset
-------------------------------
|  *  |  W  |     |  W  |     |
|(0,0)|(0,1)|(0,2)|(0,3)|(0,4)|
-------------------------------
|     |     |     |     |     |
|(1,0)|(1,1)|(1,2)|(1,3)|(1,4)|
-------------------------------
|     |     |     |     |     |
|(2,0)|(2,1)|(2,2)|(2,3)|(2,4)|
-------------------------------
|     |     |     |     |     |
|(3,0)|(3,1)|(3,2)|(3,3)|(3,4)|
-------------------------------
|     |     |     |     |  G  |
|(4,0)|(4,1)|(4,2)|(4,3)|(4,4)|
-------------------------------

action: ↓, reward: 0.0, done: False, total_steps: 1
-------------------------------
|  S  |  W  |     |  W  |     |
|(0,0)|(0,1)|(0,2)|(0,3)|(0,4)|
-------------------------------
|  *  |     |     |     |     |
|(1,0)|(1,1)|(1,2)|(1,3)|(1,4)|
-------------------------------
|     |     |     |     |     |
|(2,0)|(2,1)|(2,2)|(2,3)|(2,4)|
-------------------------------
|     |     |     |     |     |
|(3,0)|(3,1)|(3,2)|(3,3)|(3,4)|
-------------------------------
|     |     |

## Lesson 3. 랜덤 워크 환경

In [ ]:
import random
import time

# -------------------------------
# T1 0 1 2 3 4 T2
# -------------------------------


class RandomWalk():
    def __init__(
            self,
            num_internal_states=5,        # 종료 상태를 제외한 내부 상태 개수
            transition_reward=0.0,        # 일반적인 상태 전이 보상
            left_terminal_reward=0.0,     # 왼쪽 종료 상태로 이동하는 행동 
                                          # 수행 시 받는 보상
            right_terminal_reward=1.0     # 오른쪽 종료 상태로 이동하는 행동 
                                          # 수행 시 받는 보상
    ):
        self.__version__ = "0.0.1"

        self.num_internal_states = num_internal_states

        self.num_states = num_internal_states + 2
        self.STATES = [i for i in range(num_internal_states)]
        self.TERMINAL_STATES = ['T1', 'T2']

        # 모든 가능한 행동
        self.ACTION_LEFT = 0
        self.ACTION_RIGHT = 1
        self.ACTION_SYMBOLS = ["\u2190", "\u2192"]

        # 종료 상태를 제외한 임의의 상태에서 왼쪽 이동 또는 오른쪽 이동
        self.ACTIONS = [
            self.ACTION_LEFT,
            self.ACTION_RIGHT
        ]
        self.NUM_ACTIONS = len(self.ACTIONS)

        # 시작 상태 위치
        self.START_STATE = self.STATES[int(num_internal_states / 2)]

        self.transition_reward = transition_reward

        self.left_terminal_reward = left_terminal_reward

        self.right_terminal_reward = right_terminal_reward

        self.current_state = None

    def reset(self):
        self.current_state = self.START_STATE
        return self.current_state

    def moveto(self, state):
        self.current_state = state

    def get_next_state(self, state, action):
        if state in self.TERMINAL_STATES:
            next_state = state
        else:
            if action == self.ACTION_LEFT:
                if state == 0:
                    next_state = 'T1'
                else:
                    next_state = state - 1
            elif action == self.ACTION_RIGHT:
                if state == self.num_internal_states - 1:
                    next_state = 'T2'
                else:
                    next_state = state + 1
            else:
                raise ValueError()

        return next_state

    def get_reward(self, state, next_state):
        if next_state == 'T1':
            reward = self.left_terminal_reward
        elif next_state == 'T2':
            reward = self.right_terminal_reward
        else:
            reward = self.transition_reward

        return reward

    def get_state_action_probability(self, state, action):
        next_state = self.get_next_state(state, action)

        reward = self.get_reward(state, next_state)
        prob = 1.0

        return next_state, reward, prob

    # take @action in @state
    # @return: (reward, new state)
    def step(self, action):
        next_state = self.get_next_state(
            state=self.current_state, action=action
        )

        reward = self.get_reward(self.current_state, next_state)

        self.current_state = next_state

        if self.current_state in self.TERMINAL_STATES:
            done = True
        else:
            done = False

        return next_state, reward, done, None

    def render(self, mode='human'):
        print(self.__str__(), end="\n\n")

    def get_random_action(self):
        return random.choice(self.ACTIONS)

    def __str__(self):
        randomwalk_str = ""
        randomwalk_str += " T1 " + " ".join(
            ["{0}".format(i) for i in range(self.num_internal_states)]
        ) + " T2\n"

        if self.current_state in self.STATES:
            blank = "    " + "  " * self.current_state
        elif self.current_state == 'T1':
            blank = " "
        elif self.current_state == 'T2':
            blank = "  " + "  " * (self.num_internal_states + 1)
        else:
            raise ValueError()

        randomwalk_str += blank + "*"

        return randomwalk_str

In [ ]:
def main():
    env = RandomWalk()
    env.reset()
    print("reset")
    env.render()

    done = False
    total_steps = 0
    while not done:
        total_steps += 1
        action = env.get_random_action()
        next_state, reward, done, _ = env.step(action)
        print("action: {0}, reward: {1}, done: {2}, total_steps: {3}".format(
            env.ACTION_SYMBOLS[action],
            reward, done, total_steps
        ))
        env.render()

        time.sleep(1)

In [ ]:
if __name__ == "__main__":
    main()

reset
 T1 0 1 2 3 4 T2
        *

action: →, reward: 0.0, done: False, total_steps: 1
 T1 0 1 2 3 4 T2
          *

action: ←, reward: 0.0, done: False, total_steps: 2
 T1 0 1 2 3 4 T2
        *

action: ←, reward: 0.0, done: False, total_steps: 3
 T1 0 1 2 3 4 T2
      *

action: →, reward: 0.0, done: False, total_steps: 4
 T1 0 1 2 3 4 T2
        *

action: ←, reward: 0.0, done: False, total_steps: 5
 T1 0 1 2 3 4 T2
      *

action: →, reward: 0.0, done: False, total_steps: 6
 T1 0 1 2 3 4 T2
        *

action: →, reward: 0.0, done: False, total_steps: 7
 T1 0 1 2 3 4 T2
          *

action: ←, reward: 0.0, done: False, total_steps: 8
 T1 0 1 2 3 4 T2
        *

action: ←, reward: 0.0, done: False, total_steps: 9
 T1 0 1 2 3 4 T2
      *

action: →, reward: 0.0, done: False, total_steps: 10
 T1 0 1 2 3 4 T2
        *

action: →, reward: 0.0, done: False, total_steps: 11
 T1 0 1 2 3 4 T2
          *

action: →, reward: 0.0, done: False, total_steps: 12
 T1 0 1 2 3 4 T2
            *


## Lesson 4. 미로 환경

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 미로 내 장애물 및 시작 상태, 종료 상태 정보등을 모두 지닌 미로 클래스
import random
import time

#-------------------------------------------------------------
#|     |     |     |     |     |     |     |  x  |  x  |     |
#|(0,0)|(0,1)|(0,2)|(0,3)|(0,4)|(0,5)|(0,6)|(0,7)|(0,8)|(0,9)
#-------------------------------------------------------------
#|     |     |     |     |  x  |  x  |     |     |     |     |
#|(1,0)|(1,1)|(1,2)|(1,3)|(1,4)|(1,5)|(1,6)|(1,7)|(1,8)|(1,9)
#-------------------------------------------------------------
#|     |  S  |  x  |     |  x  |  x  |     |     |  x  |     |
#|(2,0)|(2,1)|(2,2)|(2,3)|(2,4)|(2,5)|(2,6)|(2,7)|(2,8)|(2,9)
#-------------------------------------------------------------
#|     |     |  x  |     |     |     |     |     |  x  |     |
#|(3,0)|(3,1)|(3,2)|(3,3)|(3,4)|(3,5)|(3,6)|(3,7)|(3,8)|(3,9)
#-------------------------------------------------------------
#|     |     |  x  |     |     |     |     |  x  |  x  |     |
#|(4,0)|(4,1)|(4,2)|(4,3)|(4,4)|(4,5)|(4,6)|(4,7)|(4,8)|(4,9)
#-------------------------------------------------------------
#|     |     |     |     |     |     |     |  x  |  x  |  G  |
#|(5,0)|(5,1)|(5,2)|(5,3)|(5,4)|(5,5)|(5,6)|(5,7)|(5,8)|(5,9)
#-------------------------------------------------------------

class Maze():
    def __init__(self):
        # 미로의 가로 길이
        self.MAZE_WIDTH = 10

        # 미로의 세로 길이
        self.MAZE_HEIGHT = 6

        # 모든 가능한 행동
        self.ACTION_UP = 0
        self.ACTION_DOWN = 1
        self.ACTION_LEFT = 2
        self.ACTION_RIGHT = 3
        self.ACTIONS = [
            self.ACTION_UP,
            self.ACTION_DOWN,
            self.ACTION_LEFT,
            self.ACTION_RIGHT
        ]
        
        # UP, DOWN, LEFT, RIGHT
        self.ACTION_SYMBOLS = ["↑", "↓", "←", "→"] 
        self.NUM_ACTIONS = len(self.ACTIONS)

        # 시작 상태 위치
        self.START_STATE = (2, 1)

        # 종료 상태 위치
        self.GOAL_STATES = [(5, 9)]

        # 장애물들의 위치
        self.OBSTACLES = [
            (0, 7), (0, 8),
            (1, 4), (1, 5),
            (2, 2), (2, 4), (2, 5), (2, 8),
            (3, 2), (3, 8),
            (4, 2), (4, 7), (4, 8),
            (5, 7), (5, 8)
        ]

        self.current_state = None

    def reset(self):
        self.current_state = self.START_STATE
        return self.current_state

    # take @action in @state
    # @return: [new state, reward]
    def step(self, action):
        x, y = self.current_state
        if action == self.ACTION_UP:
            x = max(x - 1, 0)
        elif action == self.ACTION_DOWN:
            x = min(x + 1, self.MAZE_HEIGHT - 1)
        elif action == self.ACTION_LEFT:
            y = max(y - 1, 0)
        elif action == self.ACTION_RIGHT:
            y = min(y + 1, self.MAZE_WIDTH - 1)

        if (x, y) in self.OBSTACLES:
            x, y = self.current_state

        if (x, y) in self.GOAL_STATES:
            reward = 1.0
        else:
            reward = 0.0

        self.current_state = (x, y)

        if self.current_state in self.GOAL_STATES:
            done = True
        else:
            done = False

        return (x, y), reward, done, None

    def render(self):
        print(self.__str__())

    def get_random_action(self):
        return random.choice(self.ACTIONS)

    def __str__(self):
        maze_str = ""
        for i in range(self.MAZE_HEIGHT):
            maze_str += "-------------------------------------------------------------\n"
            out = '| '
            for j in range(self.MAZE_WIDTH):
                if (i, j) == self.START_STATE:
                    t = "S"
                elif (i, j) in self.GOAL_STATES:
                    t = "G"
                elif self.current_state[0] == i and self.current_state[1] == j:
                    t = "*"
                else:
                    t = " " if (i, j) not in self.OBSTACLES else "x"
                out += str(" {0} ".format(t)) + ' | '
            maze_str += out + "\n"

            for j in range(self.MAZE_WIDTH):
                maze_str += "|({0},{1})".format(i, j)
            maze_str += "|\n"

        maze_str += "-------------------------------------------------------------\n"
        return maze_str

In [ ]:
def main():
    env = Maze()
    env.reset()
    print("reset")
    env.render()

    done = False
    total_steps = 0
    while not done:
        total_steps += 1
        action = env.get_random_action()
        reward, next_state, done, _ = env.step(action)
        print("action: {0}, reward: {1}, done: {2}, total_steps: {3}".format(
            env.ACTION_SYMBOLS[action],
            reward, done, total_steps
        ))
        env.render()

        time.sleep(1)

In [ ]:
if __name__ == "__main__":
    main()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

action: ←, reward: (5, 5), done: False, total_steps: 1033
-------------------------------------------------------------
|     |     |     |     |     |     |     |  x  |  x  |     | 
|(0,0)|(0,1)|(0,2)|(0,3)|(0,4)|(0,5)|(0,6)|(0,7)|(0,8)|(0,9)|
-------------------------------------------------------------
|     |     |     |     |  x  |  x  |     |     |     |     | 
|(1,0)|(1,1)|(1,2)|(1,3)|(1,4)|(1,5)|(1,6)|(1,7)|(1,8)|(1,9)|
-------------------------------------------------------------
|     |  S  |  x  |     |  x  |  x  |     |     |  x  |     | 
|(2,0)|(2,1)|(2,2)|(2,3)|(2,4)|(2,5)|(2,6)|(2,7)|(2,8)|(2,9)|
-------------------------------------------------------------
|     |     |  x  |     |     |     |     |     |  x  |     | 
|(3,0)|(3,1)|(3,2)|(3,3)|(3,4)|(3,5)|(3,6)|(3,7)|(3,8)|(3,9)|
-------------------------------------------------------------
|     |     |  x  |     |     |     |     |  x  |  x  |     | 
|(4,0)|(4,1)|(4,2)|(4,3)|(4,4)|(